In [201]:
import numpy as np
import pandas as pd
import dateutil

In [202]:
df = pd.read_csv(r'test_data\tiny_test_2.cli', header=None, skiprows=20,  
                             infer_datetime_format=True, encoding='ANSI',
                             engine='python', parse_dates=[0], skipinitialspace=True,
                             sep=None)

In [203]:
df

,0,1,2
0,05-May-22 16:32:46,1.021174,23.012627
1,05-May-22 16:32:47,1.021168,23.011930
2,05-May-22 16:32:48,1.021230,23.011249
3,05-May-22 16:32:49,1.021246,23.010487
4,05-May-22 16:32:50,1.021287,23.009835
...,...,...,...
406,05-May-22 16:39:32,0.973842,23.711551
407,05-May-22 16:39:33,0.973218,23.712359
408,05-May-22 24:00:00,0.971580,23.713123
409,06-May-22 16:39:35,0.970920,23.713935


In [204]:
#df.iloc[407,0]="aap"
df

,0,1,2
0,05-May-22 16:32:46,1.021174,23.012627
1,05-May-22 16:32:47,1.021168,23.011930
2,05-May-22 16:32:48,1.021230,23.011249
3,05-May-22 16:32:49,1.021246,23.010487
4,05-May-22 16:32:50,1.021287,23.009835
...,...,...,...
406,05-May-22 16:39:32,0.973842,23.711551
407,05-May-22 16:39:33,0.973218,23.712359
408,05-May-22 24:00:00,0.971580,23.713123
409,06-May-22 16:39:35,0.970920,23.713935


In [205]:
def convert_to_dt_robust(df,c):
    if (np.issubdtype(df[c].dtype, np.datetime64)):
        print("Column",c,"is already date/time, nothing to do")
        return df
    elif (df[c].dtype!=object):
        print("Column",c," is no date/time string")
        return df
    
    df24 = df[c].str.contains("24:00:00")
    n24 = df24.sum()
    df.loc[df24,c] = df.loc[df24,c].str.replace("24:00:00","00:00:00")
    if (n24>0):
        print("Column",c,": corrected",n24,"occurences of '24:00:00'")
    try:
        df_dt = pd.to_datetime(df[c])
    except dateutil.parser.ParserError as e:
        print("Failed to convert column",c,"to date/time: ", str(e))
        return df
    
    df_dt[df24] += pd.Timedelta(days=1)
    df[c] = df_dt

    return df

In [206]:
# The problem is that sometimes midnight is denotes as 24:00.
# Python cannot handle this, so we need to convert it manually.
def convert_to_dt_robust(df,c):
    if (np.issubdtype(df[c].dtype, np.datetime64)):
        print("Column",c,"is already date/time, nothing to do")
        return df
    elif (df[c].dtype!=object):
        print("Column",c," is no date/time string")
        return df
    
    df24 = df[c].str.contains("24:00:00")
    n24 = df24.sum()
    df.loc[df24,c] = df.loc[df24,c].str.replace("24:00:00","00:00:00")
    if (n24>0):
        print("Column",c,": corrected",n24,"occurences of '24:00:00'")
    
    df_dt = pd.to_datetime(df[c], errors='coerce')
    df_fail = df_dt.isnull()
    nfail = df_fail.sum()
    if (nfail > 0):
        print("Failed to convert",nfail,"points in column",c,"to date/time,\n", df[df_fail][c])
        return df
    
    df_dt[df24] += pd.Timedelta(days=1)
    df[c] = df_dt

    return df
    

In [207]:
# 1) The problem is that sometimes midnight is denotes as 24:00.
#    Python cannot handle this, so we need to convert it manually.
# 2) Sometimes date & time end up in different (subsequent) columns. 
#    Merge them if this is the case.
def check_datetime_cols(df):
    cols = df.columns
    to_drop = None
    for i in range(len(cols)):
        convert_to_dt_robust(df,cols[i])
        if (i>0 and
            np.issubdtype(df[cols[i]].dtype, np.datetime64) and
            np.issubdtype(df[cols[i-1]].dtype, np.datetime64)):
            print("Merging date/time columns", cols[i-1],"and",cols[i])
            date_as_str = df.iloc[:, i-1].dt.strftime('%d-%b-%y')
            time_as_str = df.iloc[:, i].dt.strftime('%H:%M:%S.%f')
            date_time = date_as_str + " " + time_as_str
            df_dt = pd.to_datetime(date_time, errors='coerce')
            df[cols[i-1]] = df_dt
            to_drop = i
    
    if (to_drop is not None):
        df.drop(cols[to_drop], axis=1, inplace=True)
        
    return df

    

In [208]:
df2 = pd.read_csv(r'test_data\tiny_44147.ASC', header=None, skiprows=20,  
                             infer_datetime_format=True, encoding='ANSI',
                             engine='python', parse_dates=[0], skipinitialspace=True,
                             sep=None)

In [209]:
df2

,0,1,2,3,4
0,2007-06-22,15:34:40,0.00277,1.175,22.279
1,2007-06-22,15:34:41,0.00305,1.175,22.280
2,2007-06-22,15:34:42,0.00333,1.175,22.281
3,2007-06-22,15:34:43,0.00361,1.174,22.281
4,2007-06-22,15:34:44,0.00388,1.176,22.282
...,...,...,...,...,...
715,2007-06-22,15:46:35,0.20138,1.159,22.680
716,2007-06-22,15:46:36,0.20166,1.158,22.681
717,2007-06-22,15:46:37,0.20194,1.160,22.681
718,2007-06-22,15:46:38,0.20222,1.160,22.681


In [210]:
df2 = check_datetime_cols(df2)

Column 0 is already date/time, nothing to do
Column 1 : corrected 1 occurences of '24:00:00'
Merging date/time columns 0 and 1
Column 2  is no date/time string
Column 3  is no date/time string
Column 4  is no date/time string


In [211]:
df2

,0,2,3,4
0,2007-06-22 15:34:40,0.00277,1.175,22.279
1,2007-06-22 15:34:41,0.00305,1.175,22.280
2,2007-06-22 15:34:42,0.00333,1.175,22.281
3,2007-06-22 15:34:43,0.00361,1.174,22.281
4,2007-06-22 15:34:44,0.00388,1.176,22.282
...,...,...,...,...
715,2007-06-22 15:46:35,0.20138,1.159,22.680
716,2007-06-22 15:46:36,0.20166,1.158,22.681
717,2007-06-22 15:46:37,0.20194,1.160,22.681
718,2007-06-22 15:46:38,0.20222,1.160,22.681


In [212]:
df = check_datetime_cols(df)

Column 0 : corrected 2 occurences of '24:00:00'
Column 1  is no date/time string
Column 2  is no date/time string


In [213]:
df

,0,1,2
0,2022-05-05 16:32:46,1.021174,23.012627
1,2022-05-05 16:32:47,1.021168,23.011930
2,2022-05-05 16:32:48,1.021230,23.011249
3,2022-05-05 16:32:49,1.021246,23.010487
4,2022-05-05 16:32:50,1.021287,23.009835
...,...,...,...
406,2022-05-05 16:39:32,0.973842,23.711551
407,2022-05-05 16:39:33,0.973218,23.712359
408,2022-05-06 00:00:00,0.971580,23.713123
409,2022-05-06 16:39:35,0.970920,23.713935
